***
### **<h1 align="center">Reconstruction d'un volume à partir de tilt series (appliqué à la carte SD)</h1>**
***

#### Importer les bibliothèques python

In [ ]:
import tomopy
import dxchange
import matplotlib
import matplotlib.pyplot as plt
# import datetime
# import time
# import cv2
import numpy as np
from imagestacks import create_animation
from IPython.core.display import HTML
# from skimage.filters import threshold_otsu

#### Selectionner la premiere image (numero 0) du dossier tilt series

In [ ]:
fname = 'C:/Users/cayez/Documents/DONNEES SIMON/python/projetMultiTomopy/tilt_0_180/carteSD120321_0000.tif'
ind = list(range(180))
print(fname)

#### Creer le stack d'images projection

In [ ]:
proj = dxchange.reader.read_tiff_stack(fname, ind)

#### Regarder le format des données proj

In [ ]:
print('Type de données proj: ', type(proj))
print('Dimension des données: ', proj.shape)
print('Intensité Pixel min: ',np.min(proj[0])) 
print('Intensité Pixel min: ',np.max(proj[0]))

#### Afficher une image du stack

In [ ]:
numero_image = 0 #Affichons par exemple l'image 0 (la 1ere image)
plt.imshow(proj[numero_image,:,:], cmap='Greys_r') #Affiche l'image choisie tous les pixels en x et tous les pixels en y
plt.show()

#### Recouper l'image pour supprimer les zones ne contenant pas d'information (les traitement qui vont suivre seront plus rapide sur une image plus légère)

In [ ]:
xmin = 350
xmax = 1100
ymin = 150
ymax = 800
plt.imshow(proj[numero_image,xmin:xmax,ymin:ymax], cmap='Greys_r') #Affiche l'image choisie , seulement les pixels choisis en y
plt.show()

#### Regarder les tailles des images

In [ ]:
print('Taille avant recoupe = ',proj[numero_image,:,:].shape,'pixels')
print('Taille après recoupe = ',proj[numero_image,xmin:xmax,ymin:ymax].shape,'pixels')
largeur_crop = proj[numero_image,xmin:xmax,ymin:ymax].shape[1]
hauteur_crop = proj[numero_image,xmin:xmax,ymin:ymax].shape[0]
xyratio = largeur_crop/hauteur_crop
print('Ratio hauteur/largeur',xyratio)

#### Si la selection est correcte, l'appliquer à l'ensemble des images

In [ ]:
nombre_images = proj.shape[0]
print(nombre_images,'images dans la série de tilt')

#construire un tableau vide 

proj_crop = np.ndarray((nombre_images,hauteur_crop,largeur_crop), dtype=np.int32)

#le remplir avec les images crop
for i in range (0,nombre_images):
    proj_crop[i] = proj[i,xmin:xmax,ymin:ymax] 

#### Vérifier sur l'ensemble des images si la recoupe est correcte

In [ ]:
anim0 = create_animation(proj_crop, 4,4/xyratio)
display(HTML(anim0.to_jshtml()))

#### Afficher la  l'image avant sous échantillonnage

In [ ]:
plt.imshow(proj_crop[numero_image,:,:], cmap='Greys_r')
plt.show()

#### Normaliser les données en utilisant l'intensité du fond
[Documentation tomopy.prep.normalize](https://mytomopy.readthedocs.io/en/latest/api/tomopy.prep.normalize.html)

In [ ]:
normbg = tomopy.prep.normalize.normalize_bg(proj_crop)

#### Regarder le format des données normbg

In [ ]:
print('Type de données proj: ', type(normbg))
print('Dimension des données: ', normbg.shape)
print('Intensité Pixel min: ',np.min(normbg[0])) 
print('Intensité Pixel min: ',np.max(normbg[0]))

#### Normaliser les valeurs des pixels avec -log (cf Beer Lambert)

In [ ]:
# norm = tomopy.prep.normalize.minus_log(proj_crop, ncore=None, out=None)
norm =tomopy.minus_log(normbg )

#### Afficher la  l'image normalisee choisie

In [ ]:


plt.imshow(norm[numero_image,:,:], cmap='Greys_r')
plt.show()

#### Regarder le format des données norm

In [ ]:
print('Type de données proj: ', type(norm))
print('Dimension des données: ', normbg.shape)
print('Intensité Pixel min: ',np.min(norm[0])) 
print('Intensité Pixel min: ',np.max(norm[0]))

#### Générer un tableau numpy contenant les angles d'acquisition des differentes images

In [ ]:
angle_debut = 0
angle_fin = 180
ang = tomopy.angles(nombre_images,angle_debut,angle_fin)

#### Définition des lignes qui seront utilisées pour déterminer le centre de rotation

In [ ]:
centre_rot_initial = 350
y1 = 73
y2 = 600

plt.axhline(y=y1,color='blue',linewidth=2)
plt.axhline(y=y2,color='blue',linewidth=2)
plt.axvline(x=centre_rot_initial,color='red',linewidth=2)

plt.imshow(norm[numero_image,:,:], cmap='Greys_r')
plt.show()

#### Il faut définir le centre de rotation. Pour cela on va faire des tests de en reconstruisant une seule image correspondant à la ligne du haut d'abord...

In [ ]:
i=1
lignes = 8
colonnes = 5

fig = plt.figure(figsize=(22,36))
fig.add_subplot(lignes, colonnes, 1)
rec_test_centre = tomopy.recon(norm[:,y1:y1+1,:], ang,center = [centre_rot_initial] , algorithm='gridrec')
plt.imshow(rec_test_centre[0],cmap='Greys_r')
plt.title(str(i))
for k in range (centre_rot_initial-20,centre_rot_initial+20):
    rec_test_centre = tomopy.recon(norm[:,y1:y1+1,:], ang,center = [k] , algorithm='gridrec')
    fig.add_subplot(lignes, colonnes, i)
    plt.imshow(rec_test_centre[0],cmap='Greys_r')
    plt.title(str(k))
    i+=1

#### ... et à la ligne du bas

In [ ]:
i=1
lignes = 8
colonnes = 5

fig = plt.figure(figsize=(22,36))
fig.add_subplot(lignes, colonnes, 1)
rec_test_centre = tomopy.recon(norm[:,y2:y2+1,:], ang,center = [centre_rot_initial] , algorithm='gridrec')
plt.imshow(rec_test_centre[0],cmap='Greys_r')
plt.title(str(i))
for k in range (centre_rot_initial-20,centre_rot_initial+20):
    rec_test_centre = tomopy.recon(norm[:,y2:y2+1,:], ang,center = [k] , algorithm='gridrec')
    fig.add_subplot(lignes, colonnes, i)
    plt.imshow(rec_test_centre[0],cmap='Greys_r')
    plt.title(str(k))
    i+=1

#### Le centre de rotation correspond à l'image la plus nette (ici 357). On peut maintenant lancer la reconstruction sur l'ensemble des lignes de l'image avec la valeur de ceentre trouvée precedemment

In [ ]:
centre = [357]
rec = tomopy.recon(norm, ang,center =  centre , algorithm='gridrec') # Reconstruct object.

#### Afficher une des images reconstruites pour vérifier

In [ ]:
plt.imshow(rec[500],cmap='gray')
plt.show()

#### Regarder le format des images reconstruites

In [ ]:
print('Type de données proj: ', type(rec))
print('Dimension des données: ', rec.shape)
print('Intensité Pixel min: ',np.min(rec[500])) 
print('Intensité Pixel min: ',np.max(rec[500]))

#### Sauver les images reconstruites

In [ ]:
nbre_im_reconstruct = rec.shape[0]

dossier_reconstruct = 'C:/Users/cayez/Documents/DONNEES SIMON/LPCNO/Microscopie/Tomo TEM/tils series cart sd/reconstruction tomopy/'

for i in range (nbre_im_reconstruct):
    chemin_reconstruct = dossier_reconstruct+'rec'+str(i)+'.tiff'
    matplotlib.image.imsave(chemin_reconstruct, rec[i],vmin=-0.002,vmax=0.04, cmap='gray')# ici il faut selectionner les valeurs maxi des pixels

print(nbre_im_reconstruct, ' images reconstruites')   

#### Il ne reste plus qu'a visualiser dans Tomviz